In [4]:
import pandas as pd
from pulp import *
PATH = 'data/'

current_df = pd.read_csv(PATH + '2021-22/gws/merged_gw.csv', index_col=False)
gw_35 = current_df[current_df.GW == 35]


GW_dict = {}
GWs = current_df.GW.unique()
             
# Solving for the first x weeks, first part before free hit
players = current_df[current_df.GW == 1].name.unique()
for GW in sorted(GWs)[:6]:
    print(GW)
    player_price_dict = {} # key: player, value: (name, team, position, price, total points)
    print("Number players in week {} is {}".format(GW, len(players)))
    for i, player in enumerate(players):
        player_entries = current_df[current_df.name == player]
        total_points = player_entries[player_entries.GW <= GW]['total_points'].sum()
        player_team = player_entries[player_entries.GW == 1]['team'].values[0]
        player_price = player_entries[player_entries.GW == 1]['value'].values[0]
        player_position = player_entries[player_entries.GW == 1]['position'].values[0]
        player_name = player_entries[player_entries.GW == 1]['name']
        if player not in player_price_dict:
                val = (player,player_team, player_position, player_price, total_points)
                player_price_dict[player] = val
    GW_dict[GW] = player_price_dict
        #print("Player {} had {} price total in all {} played.".format(player, player_price, len(player_entries)))

gw_35.head()
data = gw_35[['name', 'team', 'position', 'total_points', 'value']]        
data.head()

# Helper variables
POS = data.position.unique()
CLUBS = data.team.unique()

BUDGET = 1000
pos_available = {
'DEF': 5,
'FWD': 3,
'MID': 5,
'GK': 2,
}

# Initialize Variables
solver_dict = {}
points_dict = {}
each_team = {}
budget_add = 0;
numbers = 0
for GW in GWs[:6]:
    print(GW)
    cur_player_price_dict = GW_dict[GW]
    names, teams, positions, prices, points = list(zip(*cur_player_price_dict.values()))
    players = [LpVariable("player_" + str(i), cat="Binary") for i in range(len(names))]
    # Initialize the problem
    prob = LpProblem("FPL_Player_Choices", LpMaximize)
    # Define the objective
    prob += lpSum(players[i] * points[i] for i in range(len(names))) # Objective
    # Build the constraints
    prob += lpSum(players[i] * prices[i] for i in range(len(names))) <= BUDGET # Budget Limit

    for pos in POS:
        prob += lpSum(players[i] for i in range(len(names)) if positions[i] == pos) <= pos_available[pos] # Position Limit

    for club in CLUBS:
        prob += lpSum(players[i] for i in range(len(names)) if teams[i] == club) <= 3 # Club Limit
    # Solve the problem
    prob.solve()
    for v in prob.variables():
        if v.varValue != 0:
            index = int(v.name.split("_")[1])
            name = names[index]
            club = teams[index]
            position = positions[index]
            point = points[index]
            price = prices[index]
            solver_dict[name] = (name,club,position,price,point)
            numbers += 1
            # print(name, position, club, point, price, sep=" | ")
            score = str(prob.objective)
            constraint = [str(const) for const in prob.constraints.values()][0]
    for v in prob.variables():
        score = score.replace(v.name, str(v.varValue))
        constraint = constraint.replace(v.name, str(v.varValue))

    score_pretty = " + ".join( re.findall('[0-9\.]*\*1.0', score) )
    constraint_pretty = " + ".join( re.findall('[0-9\.]*\*1.0', constraint) )

    points_dict[GW] = value(prob.objective)
    each_team[GW] = solver_dict
#     names2, prices2 = list(zip(*solver_dict.values()))
#     print(names2)
    solver_dict = {}
    numbers = 0
    
print(points_dict)
print(each_team)


# Solving for the best teams from each x week until end, (second half)
for GW in sorted(GWs)[:6]:
    if GW == 1:
        current_team = each_team[GW]
    else:
        current_team = each_team[GW-1]
    print("current team")
    print(current_team)
    c_names, c_teams, c_positions, c_prices, c_points = list(zip(*current_team.values()))
    player_price_dict = {} # key: player, value: (name, team, position, price, total points)
    players = current_df[current_df.GW == GW].name.unique()
    for i, player in enumerate(players):
        player_entries = current_df[current_df.name == player]
        total_points = player_entries[player_entries.GW >= GW]['total_points'].sum()
        player_team = player_entries[player_entries.GW == GW]['team'].values[0]
        player_price = player_entries[player_entries.GW == GW]['value'].values[0]
        player_position = player_entries[player_entries.GW == GW]['position'].values[0]
        player_name = player_entries[player_entries.GW == GW]['name']
        if player not in player_price_dict:
                val = (player,player_team, player_position, player_price, total_points)
                if player in current_team:
                    c_name, c_team, c_position, c_price, c_point = current_team[player];
                    val = (player,player_team, player_position, c_price, total_points)
                player_price_dict[player] = val             
    GW_dict[GW] = player_price_dict
        #print("Player {} had {} price total in all {} played.".format(player, player_price, len(player_entries)))


BUDGET = 1000
pos_available = {
'DEF': 5,
'FWD': 3,
'MID': 5,
'GK': 2,
}

# Initialize Variables
solver_dict = {}
points_dict = {}
each_team = {}
numbers = 0
for GW in GWs[:6]:
    print(GW)
    cur_player_price_dict = GW_dict[GW]
    names, teams, positions, prices, points = list(zip(*cur_player_price_dict.values()))
    players = [LpVariable("player_" + str(i), cat="Binary") for i in range(len(names))]
    # Initialize the problem
    prob = LpProblem("FPL_Player_Choices", LpMaximize)
    # Define the objective
    prob += lpSum(players[i] * points[i] for i in range(len(names))) # Objective
    # Build the constraints
    prob += lpSum(players[i] * prices[i] for i in range(len(names))) <= BUDGET # Budget Limit

    for pos in POS:
        prob += lpSum(players[i] for i in range(len(names)) if positions[i] == pos) <= pos_available[pos] # Position Limit

    for club in CLUBS:
        prob += lpSum(players[i] for i in range(len(names)) if teams[i] == club) <= 3 # Club Limit
    # Solve the problem
    prob.solve()
    for v in prob.variables():
        if v.varValue != 0:
            index = int(v.name.split("_")[1])
            name = names[index]
            club = teams[index]
            position = positions[index]
            point = points[index]
            price = prices[index]
            solver_dict[numbers] = name
            numbers += 1
            print(name, position, club, point, price, sep=" | ")
            score = str(prob.objective)
            constraint = [str(const) for const in prob.constraints.values()][0]
    for v in prob.variables():
        score = score.replace(v.name, str(v.varValue))
        constraint = constraint.replace(v.name, str(v.varValue))

    score_pretty = " + ".join( re.findall('[0-9\.]*\*1.0', score) )
    constraint_pretty = " + ".join( re.findall('[0-9\.]*\*1.0', constraint) )

    points_dict[GW] = value(prob.objective) #store the points per week
    each_team[GW] = solver_dict # stores team
    solver_dict = {}
    numbers = 0
    
print(points_dict)
print(each_team)



1
Number players in week 1 is 553
2
Number players in week 2 is 553
3
Number players in week 3 is 553
4
Number players in week 4 is 553
5
Number players in week 5 is 553
6
Number players in week 6 is 553
1
Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/kylecasey/Documents/Fantasy-Premier-League/venv/lib/python3.10/site-packages/pulp/apis/../solverdir/cbc/osx/64/cbc /var/folders/3s/js3l59qs4q73dcnvx8spqhq40000gn/T/0945b380c6e44383a6cbb0ba02ef7463-pulp.mps max timeMode elapsed branch printingOptions all solution /var/folders/3s/js3l59qs4q73dcnvx8spqhq40000gn/T/0945b380c6e44383a6cbb0ba02ef7463-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 30 COLUMNS
At line 3069 RHS
At line 3095 BOUNDS
At line 3649 ENDATA
Problem MODEL has 25 rows, 553 columns and 1659 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 190 - 0.00 seconds
Cgl0004I pro

4
Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/kylecasey/Documents/Fantasy-Premier-League/venv/lib/python3.10/site-packages/pulp/apis/../solverdir/cbc/osx/64/cbc /var/folders/3s/js3l59qs4q73dcnvx8spqhq40000gn/T/ad49b1c53a6247b0872212d6e0b97aea-pulp.mps max timeMode elapsed branch printingOptions all solution /var/folders/3s/js3l59qs4q73dcnvx8spqhq40000gn/T/ad49b1c53a6247b0872212d6e0b97aea-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 30 COLUMNS
At line 3157 RHS
At line 3183 BOUNDS
At line 3737 ENDATA
Problem MODEL has 25 rows, 553 columns and 1659 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 428.143 - 0.00 seconds
Cgl0004I processed model has 25 rows, 337 columns (337 integer (317 of which binary)) and 1011 elements
Cutoff increment increased from 1e-05 to 0.9999
Cbc0038I Initial state - 2 integers unsatisfied sum - 0.8571

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/kylecasey/Documents/Fantasy-Premier-League/venv/lib/python3.10/site-packages/pulp/apis/../solverdir/cbc/osx/64/cbc /var/folders/3s/js3l59qs4q73dcnvx8spqhq40000gn/T/a6678cc9312b4375ae9066ca1e363e70-pulp.mps max timeMode elapsed branch printingOptions all solution /var/folders/3s/js3l59qs4q73dcnvx8spqhq40000gn/T/a6678cc9312b4375ae9066ca1e363e70-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 30 COLUMNS
At line 3175 RHS
At line 3201 BOUNDS
At line 3755 ENDATA
Problem MODEL has 25 rows, 553 columns and 1659 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 508.571 - 0.00 seconds
Cgl0004I processed model has 25 rows, 358 columns (358 integer (342 of which binary)) and 1074 elements
Cutoff increment increased from 1e-05 to 0.9999
Cbc0038I Initial state - 2 integers unsatisfied sum - 0.857143

current team
{'Luke Thomas': ('Luke Thomas', 'Leicester', 'DEF', 45, 7), 'Mohamed Salah': ('Mohamed Salah', 'Liverpool', 'MID', 125, 17), 'Richarlison de Andrade': ('Richarlison de Andrade', 'Everton', 'FWD', 75, 11), 'Saïd Benrahma': ('Saïd Benrahma', 'West Ham', 'MID', 60, 12), 'Sergi Canós': ('Sergi Canós', 'Brentford', 'MID', 55, 11), 'Ethan Pinnock': ('Ethan Pinnock', 'Brentford', 'DEF', 45, 11), 'Marcos Alonso': ('Marcos Alonso', 'Chelsea', 'DEF', 55, 15), 'Ricardo Domingos Barbosa Pereira': ('Ricardo Domingos Barbosa Pereira', 'Leicester', 'DEF', 55, 12), 'Bruno Miguel Borges Fernandes': ('Bruno Miguel Borges Fernandes', 'Man Utd', 'MID', 120, 20), 'Hugo Lloris': ('Hugo Lloris', 'Spurs', 'GK', 55, 10), 'Michail Antonio': ('Michail Antonio', 'West Ham', 'FWD', 75, 13), 'Kasper Schmeichel': ('Kasper Schmeichel', 'Leicester', 'GK', 50, 9), 'Emmanuel Dennis': ('Emmanuel Dennis', 'Watford', 'FWD', 50, 12), 'Paul Pogba': ('Paul Pogba', 'Man Utd', 'MID', 75, 16), 'Trevoh Chalobah': ('T

Emmanuel Dennis | FWD | Watford | 120 | 50
Andrew Robertson | DEF | Liverpool | 175 | 70
3
Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/kylecasey/Documents/Fantasy-Premier-League/venv/lib/python3.10/site-packages/pulp/apis/../solverdir/cbc/osx/64/cbc /var/folders/3s/js3l59qs4q73dcnvx8spqhq40000gn/T/12426dbb443e4df88d0c855620556400-pulp.mps max timeMode elapsed branch printingOptions all solution /var/folders/3s/js3l59qs4q73dcnvx8spqhq40000gn/T/12426dbb443e4df88d0c855620556400-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 30 COLUMNS
At line 3363 RHS
At line 3389 BOUNDS
At line 3966 ENDATA
Problem MODEL has 25 rows, 576 columns and 1728 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 2363.4 - 0.00 seconds
Cgl0004I processed model has 25 rows, 451 columns (451 integer (450 of which binary)) and 1353 elements
Cutoff increment inc

Trent Alexander-Arnold | DEF | Liverpool | 180 | 75
Bernardo Mota Veiga de Carvalho e Silva | MID | Man City | 139 | 69
Gabriel Magalhães | DEF | Arsenal | 134 | 50
Heung-Min Son | MID | Spurs | 200 | 101
Emmanuel Dennis | FWD | Watford | 117 | 52
5
Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/kylecasey/Documents/Fantasy-Premier-League/venv/lib/python3.10/site-packages/pulp/apis/../solverdir/cbc/osx/64/cbc /var/folders/3s/js3l59qs4q73dcnvx8spqhq40000gn/T/e36b583f81cc4a97a50951e9e6b79172-pulp.mps max timeMode elapsed branch printingOptions all solution /var/folders/3s/js3l59qs4q73dcnvx8spqhq40000gn/T/e36b583f81cc4a97a50951e9e6b79172-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 30 COLUMNS
At line 3525 RHS
At line 3551 BOUNDS
At line 4157 ENDATA
Problem MODEL has 25 rows, 605 columns and 1815 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous object

{1: 2477.0, 2: 2414.0, 3: 2361.0, 4: 2300.0, 5: 2230.0, 6: 2150.0}
{1: {0: 'Bukayo Saka', 1: 'João Pedro Cavaco Cancelo', 2: 'Aymeric Laporte', 3: 'Mohamed Salah', 4: 'Jarrod Bowen', 5: 'Virgil van Dijk', 6: 'José Malheiro de Sá', 7: 'James Ward-Prowse', 8: 'Teemu Pukki', 9: 'Ivan Toney', 10: 'Trent Alexander-Arnold', 11: 'Conor Coady', 12: 'Hugo Lloris', 13: 'Heung-Min Son', 14: 'Emmanuel Dennis'}, 2: {0: 'Bukayo Saka', 1: 'João Pedro Cavaco Cancelo', 2: 'Aymeric Laporte', 3: 'Aaron Ramsdale', 4: 'Mohamed Salah', 5: 'Jarrod Bowen', 6: 'José Malheiro de Sá', 7: 'James Ward-Prowse', 8: 'Teemu Pukki', 9: 'Ivan Toney', 10: 'Trent Alexander-Arnold', 11: 'Gabriel Magalhães', 12: 'Heung-Min Son', 13: 'Emmanuel Dennis', 14: 'Andrew Robertson'}, 3: {0: 'Bukayo Saka', 1: 'João Pedro Cavaco Cancelo', 2: 'Aymeric Laporte', 3: 'Aaron Ramsdale', 4: 'Mohamed Salah', 5: 'Jarrod Bowen', 6: 'James Ward-Prowse', 7: 'José Malheiro de Sá', 8: 'Teemu Pukki', 9: 'Ivan Toney', 10: 'Trent Alexander-Arnold', 1